In [8]:
%pip install --upgrade pip
%pip install pandas
%pip install openpyxl

  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/ef/7d/500c9ad20238fcfcb4cb9243eede163594d7020ce87bd9610c9e02771876/pip-24.3.1-py3-none-any.whl.metadata
  Using cached pip-24.3.1-py3-none-any.whl.metadata (3.7 kB)
Using cached pip-24.3.1-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 23.2.1
    Uninstalling pip-23.2.1:
      Successfully uninstalled pip-23.2.1
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [9]:

import pandas as pd

# Load the dataset
data = pd.read_excel('/Users/dhruviswadia/Desktop/Visualization master Task/01 Call-Center-Dataset.xlsx', sheet_name='Sheet1')

# Clean column names
data.columns = (
    data.columns.str.strip()  # Remove leading/trailing spaces
    .str.lower()              # Convert to lowercase
    .str.replace(' ', '_')    # Replace spaces with underscores
    .str.replace('[()]', '', regex=True)  # Remove parentheses
)

# Convert 'date' and 'time' to a single datetime column
data['datetime'] = pd.to_datetime(data['date'].astype(str) + ' ' + data['time'].astype(str), errors='coerce')
data.drop(columns=['date', 'time'], inplace=True)

# Handle missing or inconsistent data
# Fill missing values in 'speed_of_answer_in_seconds' with the median
if 'speed_of_answer_in_seconds' in data.columns:
    data['speed_of_answer_in_seconds'] = data['speed_of_answer_in_seconds'].fillna(data['speed_of_answer_in_seconds'].median()).astype(int)

# Convert 'avg_talk_duration' to timedelta, handle errors
if 'avg_talk_duration' in data.columns:
    data['avg_talk_duration'] = pd.to_timedelta(data['avg_talk_duration'], errors='coerce').fillna(pd.Timedelta(seconds=0))

# Handle missing or invalid values in 'satisfaction_rating'
if 'satisfaction_rating' in data.columns:
    data['satisfaction_rating'] = data['satisfaction_rating'].fillna(data['satisfaction_rating'].median()).astype(int)

# Ensure 'answered_y/n' and 'resolved' columns have consistent values
if 'answered_y/n' in data.columns:
    data['answered_y/n'] = data['answered_y/n'].str.strip().str.upper().replace({'Y': 'Yes', 'N': 'No'})

if 'resolved' in data.columns:
    data['resolved'] = data['resolved'].str.strip().str.upper().replace({'Y': 'Yes', 'N': 'No'})

# Replace any remaining missing or invalid values with sensible defaults
for column in data.select_dtypes(include=['object']).columns:
    data[column] = data[column].fillna('Unknown')

for column in data.select_dtypes(include=['number']).columns:
    data[column] = data[column].fillna(0)

# Drop duplicates if any
data.drop_duplicates(inplace=True)

# Preview cleaned data
print(data.info())
print(data.head())

# Save cleaned data for Tableau visualization
cleaned_file_path = '/Users/dhruviswadia/Desktop/Visualization master Task/cleaned_call_center_data.xlsx'
data.to_excel(cleaned_file_path, index=False)

print(f"Cleaned data saved to {cleaned_file_path}")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   call_id                     5000 non-null   object        
 1   agent                       5000 non-null   object        
 2   topic                       5000 non-null   object        
 3   answered_y/n                5000 non-null   object        
 4   resolved                    5000 non-null   object        
 5   speed_of_answer_in_seconds  5000 non-null   int64         
 6   avgtalkduration             5000 non-null   object        
 7   satisfaction_rating         5000 non-null   int64         
 8   datetime                    5000 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(6)
memory usage: 351.7+ KB
None
  call_id    agent              topic answered_y/n resolved  \
0  ID0001    Diane   Contract related          Yes 